# ⚛️ H₃⁺ Noiseless VQE — UCC-S vs UCC-D vs UCCSD

This notebook compares three chemistry-inspired VQE ansatzes for the trihydrogen
cation **H₃⁺** using the production API:

    from vqe.core import run_vqe

Each ansatz is evaluated under identical conditions:
- Noiseless simulation (`default.qubit`)
- Optimizer: Adam
- 60 optimisation steps

Ansätze compared:
- **UCC-S** — singles only
- **UCC-D** — doubles only
- **UCCSD** — singles + doubles

For each ansatz we compute:
- Convergence curve
- Final energy
- Final statevector

This notebook is a pure client of the VQE package.

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem
import matplotlib.pyplot as plt
import os, sys

# Local imports from the installed package
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe_qpe_common.molecules import get_molecule_config
from vqe.core import run_vqe
from vqe_qpe_common.plotting import IMG_DIR
from vqe_qpe_common.molecule_viz import plot_molecule


In [ ]:
# -------------------------------------------------------------
# Molecular system: H₃⁺
# -------------------------------------------------------------
mol = get_molecule_config("H3+")

symbols = mol["symbols"]
coordinates = mol["coordinates"]
basis = mol["basis"]
charge = mol["charge"]

ham, qubits = qchem.molecular_hamiltonian(
    symbols, coordinates, charge=charge, basis=basis
)
hf = qchem.hf_state(2, qubits)

print(f"H₃⁺ → {qubits} qubits")
print("HF state =", "|" + "".join(str(int(x)) for x in hf) + "⟩")

In [ ]:
plot_molecule(
    symbols,
    coordinates,
    title="H₃⁺ (equilateral triangular geometry)",
    bonds=[(0, 1), (1, 2), (2, 0)],
    angles=[(0, 1, 2), (1, 2, 0), (2, 0, 1)],
    show_bond_lengths=True,
    show_atom_indices=False,
)
plt.show()


In [ ]:
# -------------------------------------------------------------
# Run VQE for UCC-S / UCC-D / UCCSD
# -------------------------------------------------------------
ansatzes = ["UCC-S", "UCC-D", "UCCSD"]
results = {}

for ansatz in ansatzes:
    print(f"\n▶ Running VQE with ansatz: {ansatz}")
    rec = run_vqe(
        molecule="H3+",
        ansatz_name=ansatz,
        optimizer_name="Adam",
        steps=60,
        stepsize=0.2,
        noisy=False,
        plot=False,
        seed=0,
        force=False,
    )

    results[ansatz] = rec

In [ ]:
# -------------------------------------------------------------
# Convergence comparison
# -------------------------------------------------------------
plt.figure(figsize=(10, 6))

for ansatz in ansatzes:
    plt.plot(results[ansatz]["energies"], label=ansatz)

plt.xlabel("Iteration")
plt.ylabel("Energy (Ha)")
plt.title("H₃⁺ VQE — UCC-S vs UCC-D vs UCCSD")
plt.grid(True, alpha=0.4)
plt.legend()
plt.tight_layout()

plt.savefig(os.path.join(IMG_DIR, "H3plus_Ansatz_Comparison.png"), dpi=300)
plt.show()

In [ ]:
# -------------------------------------------------------------
# Statevector reconstruction
# -------------------------------------------------------------
def reconstruct_state(rec):
    """Return complex statevector from stored real/imag components."""
    psi_r = np.array(rec["final_state_real"], dtype=float)
    psi_i = np.array(rec["final_state_imag"], dtype=float)
    return psi_r + 1j * psi_i

threshold = 1e-2

for ansatz in ansatzes:
    print("\n" + "-"*70)
    print(f"Final state decomposition for: {ansatz}")

    psi = reconstruct_state(results[ansatz])
    nz_idx = np.where(np.abs(psi) > threshold)[0]

    terms = []
    for idx in nz_idx:
        amp = psi[idx]
        bit = f"|{idx:0{qubits}b}⟩"
        if abs(amp.imag) < 1e-6:
            term = f"{amp.real:.4f}{bit}"
        else:
            term = f"({amp.real:.4f}+{amp.imag:.4f}j){bit}"
        terms.append(term)

    print(" |ψ⟩ =", " + ".join(terms))